# NLP Assignment

#### Made by Waldorf group

## Introduction

The purpose of first part of this assignment is to gain more familiarity with some of the models seen
during the course, including firs-order HMMs, PCFGs, as well as an optimization technique, called
dual decomposition, to integrate them. The second part of the assignment will give you an idea on
how shared tasks in NLP are organized by working on a CoNLL 2002 Named Entity Recognition
(NER) shared task.

## 1 HMMs, PCFGs and Dual Decomposition

In [1]:
import pandas as pd
import numpy as np

### 1.1 POS tagger

Your first task is to implement a decoder for a first-order hidden Markov model for Arabic part-ofspeech
tagging.

#### Loading data

In [2]:
# Test_sens file
test_file = open("test_sents", "r")
test_sents = test_file.read().splitlines()

dev_sents = open("dev_sents", "r").read().splitlines()
dev_tags = open("dev_postags", "r").read().splitlines()

# Hmm_trans file
hmm_trans_df = pd.read_table('hmm_trans', header=None)
hmm_trans_df.columns = ['FROM', 'TO', 'prob']

# Hmm_emits file
hmm_emits_file = open("hmm_emits", "r").readlines()
hmm_emits = [l.split('\t') for l in hmm_emits_file]
hmm_emits_df = pd.DataFrame(hmm_emits, columns = ['label', 'word', 'prob'])
hmm_emits_df['prob'] = pd.Series([float(i) for i in hmm_emits_df['prob']])

#### Labels for unknown words

There are words in the text file which are not included in emissions file, so sometimes we don't know which labels to put for the unknown ones. To fix this problem we decided to calculate the tag for an unseen word as a the most probable part-of-speech which follows the most probable label for the previous word.

For example, if current word is a new one that we've not seen before, and the one before is a noun, according to the transmission table the verb is the most probable to follow the noun, so the current word will be givel verb label. 

#### Functions

In [ ]:
def score_1(string, trans_df, emits_df):
    string = string.split()[0]
    
    labels_set = emits_df['label'].unique()
    labels_count = len(labels_set)
    
    score_1 = []
    for label in labels_set:
        tmp = trans_df.loc[(trans_df['FROM'] == 'sentence_boundary') & (trans_df['TO'] == label)]['prob'] 
        #print(tmp.values)
        trans_val = -float('inf')
        if len(tmp) != 0:
            trans_val = float(tmp.values[0])
            
        tmp = emits_df.loc[(emits_df['label'] == label) & (emits_df['word'] == string)]
        emit_val = -float('inf')
        if len(tmp) != 0:
            #print(tmp.values)
            emit_val = float(tmp.values[0][2])

        value = trans_val + emit_val # should add instead of multiply in logspace
        score_1.append(value)

    score_1_df = pd.DataFrame(score_1, index=labels_set)
    score_1_df.columns=[string]
    #print(score_1_df)
    return score_1_df

In [ ]:
def score_i(input_str, trans_df, emits_df, score_1_df):
    input_str = input_str.split()[1:]
    
    labels_set = emits_df['label'].unique()
    
    score_i_df = pd.DataFrame(index = labels_set, columns = input_str)
    
    word_number = -1
    for string in input_str:
        word_number += 1
        #print(score_i_df.dtypes)
        score_i_df = score_i_df.astype(float).fillna(-float('inf'))
        #print(score_i_df.dtypes)
        
        if(len(emits_df.loc[emits_df['word'] == string]) == 0):
            print(string, 'is not in hmm_emits')
            
            if (word_number == 0):
                #print(score_1_df.iloc[:,0])
                prev_label = (score_1_df.iloc[:,0]).idxmax(axis=0, skipna=True)
            else:
                #print("word: " + string)
                #print(score_i_df.iloc[:,word_number-1])
                prev_label = (score_i_df.iloc[:,word_number-1]).idxmax(axis=0, skipna=True)
                #print('prev_label = ' + str(prev_label))
            
            emit_prob = trans_df[trans_df['FROM'] == prev_label]['prob'].max(axis=0, skipna=True)
            idx = trans_df[trans_df['FROM'] == prev_label]['prob'].idxmax(axis=0, skipna=True)
            label = trans_df['TO'][idx]
            
            if label == 'sentence_boundary': # sentence_boundary state has no emission and is invalid as a tag
                label = 'NOUN' # it's reasonable to assign NOUN to unknown words http://stp.lingfil.uu.se/~nivre/statmet/haulrich.pdf
                emit_prob = trans_df.loc[(trans_df['FROM'] == prev_label) & (trans_df['TO'] == label)]['prob']
            
            score_i_df.loc[label].loc[str] = emit_prob
            continue
            
        
        for label in labels_set:      
            to_compare = []
            for label_from in labels_set:
                tmp_trans = trans_df.loc[(trans_df['FROM'] == label_from) & (trans_df['TO'] == label)]['prob'] 
                trans_val = -float('inf')
                if len(tmp_trans) != 0:
                    trans_val = float(tmp_trans.values[0])
                
                current_prob = -float('inf')
                if len(tmp_trans) != 0:
                    if word_number == 0:
                        current_prob = score_1_df.loc[label_from].values[0] + tmp_trans.values[0] # add instead of multiply
                    else:
                        current_prob = score_i_df.iloc[:, word_number-1].loc[label_from] + tmp_trans.values[0] # why? -> / 100

                to_compare.append(current_prob)
            maximum = np.nanmax(to_compare)#np.abs(to_compare))                
            
            tmp_emit = emits_df.loc[(emits_df['label'] == label) & (emits_df['word'] == string)]
            emit_prob = -float('inf')
            if len(tmp_emit)!=0:
                emit_prob = float(tmp_emit.values[0][2])
                
            value = emit_prob
            if (maximum != -float('inf')):#!= 0 and maximum != -1):
                value += maximum # add instead of multiply
            
            score_i_df.loc[label].loc[string] = value
    #print(score_i_df.dtypes)
    #score_i_df = score_i_df.iloc[1].astype(float).fillna(-1)
    #print(score_i_df)
    #print(score_i_df.dtypes)
    return score_i_df

#### Test for one string

In [ ]:
hmm_trans_df = pd.read_table('hmm_trans', header=None)
hmm_trans_df.columns = ['FROM','TO', 'prob']
hmm_trans_df['prob'] = pd.Series([float(i) for i in hmm_trans_df['prob']])

hmm_emits_lines = open("hmm_emits", "r").readlines()
hmm_emits = [l.split('\t') for l in hmm_emits_lines]
hmm_emits_df = pd.DataFrame(hmm_emits, columns = ['label', 'word', 'prob'])
hmm_emits_df['prob'] = pd.Series([float(i) for i in hmm_emits_df['prob']])

test_ind = 100
string = dev_sents[test_ind] #'w >wDH * >n DbATA >myrkyyn TAlbw *T* hA b {EtrAf fwry .'

#labels_set = hmm_emits_df['label'].unique()
#labels_count = len(labels_set)

score_1_df = score_1(string, hmm_trans_df, hmm_emits_df)
score_i_df = score_i(string, hmm_trans_df, hmm_emits_df, score_1_df)

In [ ]:
total = [score_1_df, score_i_df]
#print(total)
final_df = pd.concat(total, axis=1)
#print(final_df)

final_max = final_df.max(axis=0, skipna=True).values #score_i_df.max(axis=0, skipna=True).values
final_labels = final_df.idxmax(axis=0, skipna=True).values #score_i_df.idxmax(axis=0, skipna=True).values
for l in range(len((final_max))):
    if (final_max[l] == -float('inf')):
        final_labels[l] = 'NULL_LABEL'
print(' '.join(final_labels))

print(dev_tags[test_ind])

#### Computation of score_1_df, score_i_df and final labels

In [ ]:
all_labels = []
i = 0
for sent in test_sents:
    print(i, sent)
    i += 1
    score_1_df = score_1(sent, hmm_trans_df, hmm_emits_df)
    score_i_df = score_i(sent, hmm_trans_df, hmm_emits_df, score_1_df)
    
    total = [score_1_df, score_i_df]
    final_df = pd.concat(total, axis=1)

    final_max = final_df.max(axis=0, skipna=True).values
    final_labels = final_df.idxmax(axis=0, skipna=True).values
    for l in range(len(final_max)):
        if (final_max[l] == -float('inf')):
            final_labels[l]='NULL_LABEL'
    print(' '.join(final_labels))
    all_labels.append(final_labels)

#### Save the resultat in file

In [ ]:
with open("candidate_devpostags2", "a+") as file:
    for lst in all_labels:
        line = " ".join(lst)+'\n'
        file.write(line)

### 1.2 Context free grammar parser

Your second task is to implement a parser for a Chomsky normal form probabilistic context-free
grammar (PCFG) of Arabic syntax. 

In [25]:
pcfg_lines = open('pcfg', 'r').readlines()
pcfg = [l.split('\t') for l in pcfg_lines]
pcfg_df = pd.DataFrame(pcfg, columns = ['context', 'decision', 'prob'])
pcfg_df['prob'] = pd.Series([float(i) for i in pcfg_df['prob']])

test_string = 'w >wDH * >n DbATA >myrkyyn TAlbw *T* hA b {EtrAf fwry .'

In [84]:
def CKY(s, G):
    words = s.split()
    n_words = len(words)
    
    N = G['context'].unique() # set of non-terminal symbols

    d = {el:np.full((n_words, n_words), -float('inf')) for el in N}
    bp = {el:np.full((n_words, n_words), None) for el in N}
        
    for i in range(n_words):
        unary_rules = G[G['decision'] == words[i]] # filter of unary rules to the current word

        for item, row in unary_rules.iterrows():
            Z = row['context']
            d[Z][i, i] = row['prob']
            
    G[['X', 'Y']] = G['decision'].str.split(' ', expand=True) # split the decision into the constituents    
   
    binary_rules = G[G['Y'].notnull()] # filter out every binary rule

    for l in range(1,n_words):
        print("Word " + str(l) + ' out ouf ' + str(n_words-1))
        for i in range(n_words-l):
            j = i + l
            for Z in N:
                    binary_rules_Z = binary_rules[binary_rules['context'] == Z]
                    current_max = -float('inf') # to be replaced by max from loop
                    args = None
                    
                    for item, row in binary_rules_Z.iterrows():
                        X = row['X']
                        Y = row['Y']
                        
                        for s in range(i, j):
                            if (d[X][i, s] > -float('inf')) and (d[Y][s + 1, j] > -float('inf')):
                                rule_prob = row['prob']
                                prob = rule_prob + d[X][i, s] + d[Y][s + 1, j]
                                if prob > current_max:
                                    current_max = prob
                                    args = (s, X, Y)
                                    
                    if current_max > -float('inf'):
                        d[Z][i, j] = current_max
                        bp[Z][i, j] = args
    
    return d, bp

In [ ]:
d, bp = CKY(test_string, pcfg_df)

In [85]:
def recover_tree(s, bp, i, j, Z):
    """
    Return the list of the parsed tree with back pointers.
    """
    if i == j:
        return '( ' + Z + ' ' + s.split()[i] + ' )' #[Z, s.split()[i]]
    else:
        if bp[Z][i, j] == None:
            return "N/A"
        k, X, Y = bp[Z][i, j]
        return '( ' + Z + ' ' + recover_tree(s, bp, i, k, X) + ' ' + recover_tree(s, bp, k+1, j, Y) + ' ) '

In [86]:
def find_root(s, d, pcfg_df):
    # Note: the following doesn't always give S as root. Are we supposed to find
    # most probable parse tree or most probable parse tree given S as a start symbol?

    max_prob = -float('inf')
    n_words = len(s.split())
    root = 'S'

    for Z in pcfg_df['context'].unique(): # retrieve root of the most likely parse tree
        prob = d[Z][0, n_words-1]
        if prob > max_prob:
            max_prob = prob
            root = Z
            
    if root != 'S':
        print('Root not S: ' + root)
    return root

In [58]:
text_file = open("candidate_parses", "w")
text_file.write(recover_tree(string, bp4, 0, n_words-1, 'S'))
text_file.close()

In [54]:
print(recover_tree(test_string, bp, 0, len(test_string)-1, 'S'))
print('\n')
print('( S ( CC w ) ( VP-unitedchildren-PUNC ( VP ( PV >wDH ) ( NP-unitedchildren-SBAR ( NONE * ) ( SBAR ( SUB_CONJ >n ) ( S ( NP ( NOUN DbATA ) ( ADJ >myrkyyn ) ) ( VP ( PV TAlbw ) ( NP-unitedchildren-NP-unitedchildren-PP ( NONE *T* ) ( NP-unitedchildren-PP ( PRP hA ) ( PP ( IN b ) ( NP ( NOUN {EtrAf ) ( ADJ fwry ) ) ) ) ) ) ) ) ) ) ( PUNC . ) ) )')

( S ( CC w ) ( VP-unitedchildren-PUNC ( VP ( PV >wDH ) ( NP-unitedchildren-SBAR ( NONE * ) ( SBAR ( SUB_CONJ >n ) ( S ( NP ( NOUN DbATA ) ( ADJ >myrkyyn ) )  ( VP ( PV TAlbw ) ( NP-unitedchildren-NP-unitedchildren-PP ( NONE *T* ) ( NP-unitedchildren-PP ( PRP hA ) ( PP ( IN b ) ( NP ( NOUN {EtrAf ) ( ADJ fwry ) )  )  )  )  )  )  )  )  )  ( PUNC . ) )  ) 


( S ( CC w ) ( VP-unitedchildren-PUNC ( VP ( PV >wDH ) ( NP-unitedchildren-SBAR ( NONE * ) ( SBAR ( SUB_CONJ >n ) ( S ( NP ( NOUN DbATA ) ( ADJ >myrkyyn ) ) ( VP ( PV TAlbw ) ( NP-unitedchildren-NP-unitedchildren-PP ( NONE *T* ) ( NP-unitedchildren-PP ( PRP hA ) ( PP ( IN b ) ( NP ( NOUN {EtrAf ) ( ADJ fwry ) ) ) ) ) ) ) ) ) ) ( PUNC . ) ) )


In [ ]:
all_parses = []
i = 0
for i, sent in enumerate(reversed(test_sents)):
    print(i, sent)
    n_words = len(sent.split())
    d, bp = CKY(sent, pcfg_df)
    Z = find_root(sent, d, pcfg_df)
    print(recover_tree(sent, bp, 0, n_words-1, Z))
    res = recover_tree(sent, bp, 0, n_words-1, 'S')
    print(res)
    all_parses.append(res)

0 lyt hA kAnt * HADrp !
Word 1 out ouf 5
Word 2 out ouf 5
Word 3 out ouf 5
Word 4 out ouf 5
Word 5 out ouf 5
Root not S: PSEUDO_VERB-unitedchildren-NP-unitedchildren-S-unitedchildren-PUNC
( PSEUDO_VERB-unitedchildren-NP-unitedchildren-S-unitedchildren-PUNC ( PSEUDO_VERB lyt ) ( NP-unitedchildren-S-unitedchildren-PUNC ( PRP hA ) ( S-unitedchildren-PUNC ( VP ( PV kAnt ) ( NP-unitedchildren-ADJP ( NONE * ) ( ADJ HADrp ) )  )  ( PUNC ! ) )  )  ) 
( S ( VP ( PSEUDO_VERB lyt ) ( NP-unitedchildren-S ( PRP hA ) ( VP ( PV kAnt ) ( NP-unitedchildren-ADJP ( NONE * ) ( ADJ HADrp ) )  )  )  )  ( PUNC ! ) ) 
1 lkn AlqSp lyst * mtwqfp ElY mA yryd Almstr blyks *T* >w *0* yrfD * *T* ,
Word 1 out ouf 16
Word 2 out ouf 16
Word 3 out ouf 16
Word 4 out ouf 16
Word 5 out ouf 16
Word 6 out ouf 16
Word 7 out ouf 16
Word 8 out ouf 16
Word 9 out ouf 16
Word 10 out ouf 16
Word 11 out ouf 16
Word 12 out ouf 16
Word 13 out ouf 16
Word 14 out ouf 16
Word 15 out ouf 16
Word 16 out ouf 16
Root not S: PSEUDO_VERB-unit

Word 1 out ouf 12
Word 2 out ouf 12
Word 3 out ouf 12
Word 4 out ouf 12
Word 5 out ouf 12
Word 6 out ouf 12
Word 7 out ouf 12
Word 8 out ouf 12
Word 9 out ouf 12
Word 10 out ouf 12
Word 11 out ouf 12
Word 12 out ouf 12
Root not S: CC-unitedchildren-VP-unitedchildren-PUNC
( CC-unitedchildren-VP-unitedchildren-PUNC ( CC w ) ( VP-unitedchildren-PUNC ( VP ( PV SrH ) ( NP-unitedchildren-SBAR ( NNP ErfAt ) ( SBAR ( SUB_CONJ >n ) ( S ( NP ( NNP AllqA' ) ( CC-unitedchildren-NNP ( CC w ) ( NNP slymAn ) )  )  ( PUNC-unitedchildren-ADJP-unitedchildren-NP-unitedchildren-PUNC ( PUNC " ) ( ADJP-unitedchildren-NP-unitedchildren-PUNC ( ADJ mhm ) ( NP-unitedchildren-PUNC ( NP ( NOUN jdA ) ( NOUN jdA ) )  ( PUNC " ) )  )  )  )  )  )  )  ( PUNC . ) )  ) 
( S ( CC w ) ( VP-unitedchildren-PUNC ( VP ( PV SrH ) ( NP-unitedchildren-SBAR ( NNP ErfAt ) ( SBAR ( SUB_CONJ >n ) ( S ( NP ( NNP AllqA' ) ( CC-unitedchildren-NNP ( CC w ) ( NNP slymAn ) )  )  ( PUNC-unitedchildren-ADJP-unitedchildren-NP-unitedchildren-

Word 1 out ouf 6
Word 2 out ouf 6
Word 3 out ouf 6
Word 4 out ouf 6
Word 5 out ouf 6
Word 6 out ouf 6
N/A
N/A
24 h*A byt syAsy kbyr
Word 1 out ouf 3
Word 2 out ouf 3
Word 3 out ouf 3
Root not S: PRN-unitedchildren-NP
( PRN-unitedchildren-NP ( DT h*A ) ( NP ( NOUN byt ) ( ADJ-unitedchildren-ADJ ( ADJ syAsy ) ( ADJ kbyr ) )  )  ) 
( S ( DT h*A ) ( NP ( NOUN byt ) ( ADJ-unitedchildren-ADJ ( ADJ syAsy ) ( ADJ kbyr ) )  )  ) 
25 Antbh * .
Word 1 out ouf 2
Word 2 out ouf 2
N/A
N/A
26 w >qdr * jhwd Aldktwr bTrs gAly w jhwd AlHkwmp AllbnAnyp fy h*A Al<TAr " .
Word 1 out ouf 15
Word 2 out ouf 15
Word 3 out ouf 15
Word 4 out ouf 15
Word 5 out ouf 15
Word 6 out ouf 15
Word 7 out ouf 15
Word 8 out ouf 15
Word 9 out ouf 15
Word 10 out ouf 15
Word 11 out ouf 15
Word 12 out ouf 15
Word 13 out ouf 15
Word 14 out ouf 15
Word 15 out ouf 15
Root not S: CC-unitedchildren-S-unitedchildren-CC-unitedchildren-S-unitedchildren-PUNC-unitedchildren-PUNC
( CC-unitedchildren-S-unitedchildren-CC-unitedchildren-S-un

Word 14 out ouf 17
Word 15 out ouf 17
Word 16 out ouf 17
Word 17 out ouf 17
N/A
N/A
35 Hdd * l y mwEdA l *0* >xlE * jsd y ElY EtbAt rHmt k ,
Word 1 out ouf 15


In [87]:
all_parses = []
for i, sent in enumerate(dev_sents):
    print(i, sent)
    n_words = len(sent.split())
    d, bp = CKY(sent, pcfg_df)
    Z = find_root(sent, d, pcfg_df)
    print(recover_tree(sent, bp, 0, n_words-1, Z))
    res = recover_tree(sent, bp, 0, n_words-1, 'S')
    print(res)
    all_parses.append(res)

0 w >wDH * >n DbATA >myrkyyn TAlbw *T* hA b {EtrAf fwry .
Word 1 out ouf 12
Word 2 out ouf 12
Word 3 out ouf 12
Word 4 out ouf 12
Word 5 out ouf 12
Word 6 out ouf 12
Word 7 out ouf 12
Word 8 out ouf 12
Word 9 out ouf 12
Word 10 out ouf 12
Word 11 out ouf 12
Word 12 out ouf 12
Root not S: CC-unitedchildren-S-unitedchildren-PP-unitedchildren-PUNC
( CC-unitedchildren-S-unitedchildren-PP-unitedchildren-PUNC ( CC w ) ( S-unitedchildren-PP-unitedchildren-PUNC ( VP ( PV >wDH ) ( NP-unitedchildren-SBAR ( NONE * ) ( SBAR ( SUB_CONJ >n ) ( S ( NP ( NOUN DbATA ) ( ADJ >myrkyyn ) )  ( VP ( PV TAlbw ) ( NP-unitedchildren-NP ( NONE *T* ) ( PRP hA ) )  )  )  )  )  )  ( PP-unitedchildren-PUNC ( PP ( IN b ) ( NP ( NOUN {EtrAf ) ( ADJ fwry ) )  )  ( PUNC . ) )  )  ) 
( S ( CC w ) ( VP-unitedchildren-PUNC ( VP ( PV >wDH ) ( NP-unitedchildren-SBAR ( NONE * ) ( SBAR ( SUB_CONJ >n ) ( S ( NP ( NOUN DbATA ) ( ADJ >myrkyyn ) )  ( VP ( PV TAlbw ) ( NP-unitedchildren-NP-unitedchildren-PP ( NONE *T* ) ( NP-unite

KeyboardInterrupt: 

In [95]:
for i, sent in enumerate(test_sents[30:46]):
    print(i, sent)
    n_words = len(sent.split())
    d, bp = CKY(sent, pcfg_df)
    Z = find_root(sent, d, pcfg_df)
    print(recover_tree(sent, bp, 0, n_words-1, Z))
    res = recover_tree(sent, bp, 0, n_words-1, 'S')
    print(res)
    all_parses.append(res)

0 " ywm flsTyn " fy Almdynp AlryADyp H$d *T* 20 >lfA
Word 1 out ouf 10
Word 2 out ouf 10
Word 3 out ouf 10
Word 4 out ouf 10
Word 5 out ouf 10
Word 6 out ouf 10
Word 7 out ouf 10
Word 8 out ouf 10
Word 9 out ouf 10
Word 10 out ouf 10
Root not S: NP-unitedchildren-PP-unitedchildren-NP-unitedchildren-NP-unitedchildren-NP
( NP-unitedchildren-PP-unitedchildren-NP-unitedchildren-NP-unitedchildren-NP ( NP ( PUNC " ) ( NOUN-unitedchildren-NP-unitedchildren-PUNC ( NOUN ywm ) ( NP-unitedchildren-PUNC ( NOUN_PROP flsTyn ) ( PUNC " ) )  )  )  ( PP-unitedchildren-NP-unitedchildren-NP-unitedchildren-NP ( PP ( IN fy ) ( NP ( NOUN Almdynp ) ( ADJ AlryADyp ) )  )  ( NP-unitedchildren-NP-unitedchildren-NP ( NOUN H$d ) ( NP-unitedchildren-NP ( NONE *T* ) ( NP ( NOUN_NUM 20 ) ( NOUN_NUM >lfA ) )  )  )  )  ) 
( S ( NP ( NP ( PUNC " ) ( NOUN-unitedchildren-NP-unitedchildren-PUNC ( NOUN ywm ) ( NP-unitedchildren-PUNC ( NOUN_PROP flsTyn ) ( PUNC " ) )  )  )  ( PP ( IN fy ) ( NP ( NOUN Almdynp ) ( ADJ AlryADy

Word 1 out ouf 19
Word 2 out ouf 19
Word 3 out ouf 19
Word 4 out ouf 19
Word 5 out ouf 19
Word 6 out ouf 19
Word 7 out ouf 19
Word 8 out ouf 19
Word 9 out ouf 19
Word 10 out ouf 19
Word 11 out ouf 19
Word 12 out ouf 19
Word 13 out ouf 19
Word 14 out ouf 19
Word 15 out ouf 19
Word 16 out ouf 19
Word 17 out ouf 19
Word 18 out ouf 19
Word 19 out ouf 19
Root not S: IV_PASS-unitedchildren-NP-unitedchildren-NP-unitedchildren-NP-unitedchildren-PP-unitedchildren-PUNC
( IV_PASS-unitedchildren-NP-unitedchildren-NP-unitedchildren-NP-unitedchildren-PP-unitedchildren-PUNC ( IV_PASS y*kr ) ( NP-unitedchildren-NP-unitedchildren-NP-unitedchildren-PP-unitedchildren-PUNC ( NP ( NOUN >xyrA ) ( SBAR ( SUB_CONJ >n ) ( S ( NOUN_PROP lbnAn ) ( VP ( IV ystDyf ) ( NP-unitedchildren-PP ( NONE *T* ) ( PP ( IN fy ) ( NP ( NOUN t$ryn ) ( ADJ_NUM-unitedchildren-NOUN_NUM ( ADJ_NUM Al>wl ) ( NOUN_NUM 2003 ) )  )  )  )  )  )  )  )  ( NP-unitedchildren-NP-unitedchildren-PP-unitedchildren-PUNC ( NP ( NOUN Alm&tmr ) ( AD

In [ ]:
with open("candidate_testparses", "a+") as file:
    for lst in all_parses:
        line = lst + '\n'
        file.write(line)

### 1.3 Dual decomposition - not solved

Given input sentence x, it is required to find the highest scoring derivation (1).

In [ ]:
def find_all_roots(s, d, pcfg_df):
    # Note: the following doesn't always give S as root. Are we supposed to find
    # most probable parse tree or most probable parse tree given S as a start symbol?

    limit = -float('inf')
    n_words = len(s.split())
    roots = []

    for Z in pcfg_df['context'].unique(): # retrieve root of the most likely parse tree
        prob = d[Z][0, n_words-1]
        if prob > limit:
            roots.append(Z)
            
    return roots

In [ ]:
def dual_decomposition(s, emits_df, pcfg_df, K = 5):
    n_words = len(s.split())
    POS_set = emits_df['label'].unique()
    POS_size = len(POS_set)
    u = np.zeros(n_words, POS_size)

    d, bp = CKY(s, pcfg_df)
    roots = find_all_roots(s, d, pcfg_df)
    
    for k in range(K):
        y_max = -float('inf')
        y = None
        for root in roots:
            prob = d[root][0, n_words-1]
            sum = 0
            for i in range(n_words):
                for t in range(POS_size):
                    sum += u[i, t]* # y(i,t) = 1 iff parse tree y has a tag t ∈ T at position i - not sure how to implement with our design...

## 2 Named Entity Recognition